In [1]:
import pyarrow.parquet as pq
import pyarrow as pa
import os

# Directory containing the partial .parquet files
directory = "/mnt/carpedkm_data/image_gen_ds/relaion2B-en-research"

# List all .snappy.parquet files
parquet_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith(".snappy.parquet")]

# # Merge the partial files into one
# tables = [pq.read_table(f) for f in parquet_files]
# combined_table = pa.concat_tables(tables)

# # Write the combined table to a new file
# pq.write_table(combined_table, "merged_file.snappy.parquet")

In [3]:
import pandas as pd

In [4]:
df = pd.read_parquet(parquet_files[0])

In [5]:
print(df.columns)  

Index(['url', 'similarity', 'hash', 'pwatermark', 'punsafe', 'caption', 'key',
       'status', 'error_message', 'width', 'height', 'original_width',
       'original_height', 'exif', 'md5'],
      dtype='object')


In [6]:
# Example: Filter for safe images with no watermark
filtered_df = df[(df['punsafe'] < 0.5) & (df['pwatermark'] < 0.5)]
print(f"Filtered dataset size: {len(filtered_df)}")

Filtered dataset size: 13115819


In [7]:
urls = filtered_df['url'].tolist()[:1000]  # Example: Limit to 1000 images for testing

In [8]:
chunk_size = 100_000  # Example chunk size
for i, chunk in enumerate(range(0, len(df), chunk_size)):
    chunk_df = df.iloc[chunk:chunk + chunk_size]
    chunk_df.to_parquet(f"chunk_{i}.parquet")

KeyboardInterrupt: 

In [ ]:
import crcmod
import binascii

# Compute and verify CRC for a .parquet file
def verify_crc(parquet_file, crc_file):
    crc32_func = crcmod.predefined.mkCrcFun('crc-32')
    with open(parquet_file, "rb") as f:
        file_data = f.read()
        calculated_crc = crc32_func(file_data)

    with open(crc_file, "rb") as f:
        stored_crc = int(f.read().strip(), 16)  # CRC values are often in hexadecimal

    return calculated_crc == stored_crc

# Example
parquet_file = "partial_file.snappy.parquet"
crc_file = "partial_file.snappy.parquet.crc"
if verify_crc(parquet_file, crc_file):
    print(f"{parquet_file} is valid.")
else:
    print(f"{parquet_file} is corrupted.")

In [ ]:
import crcmod
import binascii

# Compute and verify CRC for a .parquet file
def verify_crc(parquet_file, crc_file):
    crc32_func = crcmod.predefined.mkCrcFun('crc-32')
    with open(parquet_file, "rb") as f:
        file_data = f.read()
        calculated_crc = crc32_func(file_data)

    with open(crc_file, "rb") as f:
        stored_crc = int(f.read().strip(), 16)  # CRC values are often in hexadecimal

    return calculated_crc == stored_crc

# Example
parquet_file = "partial_file.snappy.parquet"
crc_file = "partial_file.snappy.parquet.crc"
if verify_crc(parquet_file, crc_file):
    print(f"{parquet_file} is valid.")
else:
    print(f"{parquet_file} is corrupted.")

In [ ]:
import pyarrow.parquet as pq

table = pq.read_table("merged_file.snappy.parquet")
print(table.to_pandas())  # Convert to pandas DataFrame

In [ ]:
import pandas as pd

df = pd.read_parquet("merged_file.snappy.parquet")
print(df.head())